### 0. Install and Import Dependencies¶


In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib


  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

aiohttp 3.6.3 requires yarl<1.6.0,>=1.0, but you'll have yarl 1.6.2 which is incompatible.


  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=82b176c06ee9509aba1202ab28f585e698487a205f17e3bd09f504cb1beadec5
  Stored in directory: c:\users\youss\appdata\local\pip\cache\wheels\62\76\4c\aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: h5py
    Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.31.0
    Uninstalling grpcio-1.31.0:
      Successfully uninstalled grpcio-1.31.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: typing-extensions
    Found existing installation: 

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2


### 1. Load model 

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()


### 2. Make Detection 

In [8]:
    img = frame.copy()


In [21]:
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [139]:
# acessing web cam with openCV
cap = cv2.VideoCapture(0)
squat_counter = 0
count_bool = True 
# reading the video frame from the web cam
while cap.isOpened():
    
    frame_counter = frame_counter + 1   
    # read the frames from the web cam
    ret, frame = cap.read()
    
    # reshape image to be in proper format of the model
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    # setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # make predicitions
    
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    # get the results
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.35)
    draw_keypoints(frame, keypoints_with_scores, 0.35)
    squat_counter, count_bool = count_squats(frame, keypoints_with_scores, 0.35, squat_counter, count_bool)        
    
    cv2.imshow('MoveNet Lighting', frame)
    
    # Stop reading if q is pressed
    if cv2.waitKey(10) & 0xff == ord('q'):
        break 

        
cap.release()
cv2.destroyAllWindows()
        

[[[[0.7253021  0.6170354  0.36016083]
   [0.6139716  0.70847356 0.5488645 ]
   [0.6256349  0.5263308  0.5059119 ]
   [0.62624127 0.8010723  0.5767627 ]
   [0.6379711  0.426535   0.6729853 ]
   [0.80532664 0.85797894 0.40624303]
   [0.8078901  0.24423335 0.3162533 ]
   [0.96589935 0.95094055 0.05583251]
   [0.9580404  0.18306093 0.02797416]
   [0.9766556  0.8755388  0.05815342]
   [0.96827996 0.4144089  0.00756946]
   [0.67482734 0.7857176  0.01125687]
   [0.59503347 0.36722574 0.00903526]
   [0.980287   0.9662236  0.01704109]
   [0.8217358  0.3012392  0.00383985]
   [0.7713253  0.8593969  0.00555959]
   [0.66707325 0.40406233 0.00937682]]]]
[[[[0.72939724 0.6123998  0.58304375]
   [0.621756   0.7045552  0.4200849 ]
   [0.6310297  0.5223709  0.62619656]
   [0.6308757  0.80252373 0.63678336]
   [0.6413478  0.4204716  0.5174869 ]
   [0.78785056 0.8562464  0.4291919 ]
   [0.8047871  0.22935162 0.36045268]
   [0.967206   0.95279694 0.03957856]
   [0.9538822  0.11677185 0.02511147]
   [0.975

In [127]:
# example of the results 
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]


In [29]:
# the coordinates of the left elbow 
np.array((left_elbow[:2]*[480,460]).astype(int))

array([466, 444])

### 3. Draw Keypoints

In [30]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)
    


### 4. Draw Edges 

In [33]:
# The differnet connections between the coordinates
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [34]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

### 5. Detect Squats 

In [140]:
def count_squats(frame, keypoints, confidence_threshold, squat_counter, count_bool ):
    squat_frame_new = 0
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    # extract the y,x,c for the left and right hip and left and right knee 
    y_left_hip,x_left_hip,c_left_hip = shaped[11]
    y_right_hip,x_right_hip,c_righ_hip = shaped[12]
    
    y_left_knee,x_left_knee,c_left_knee = shaped[13]
    y_right_knee,x_right_knee,c_righ_knee = shaped[14]

    if count_bool:
        if (c_left_hip > confidence_threshold) & (c_righ_hip > confidence_threshold) & (c_left_knee > confidence_threshold) & (c_righ_knee > confidence_threshold):  
            if (y_left_hip >= y_left_knee -20 )  & (y_right_hip >= y_right_knee-20):
                squat_counter = squat_counter + 1
                count_bool = False
    else:
        if (y_left_hip + 50 <= y_left_knee )  & (y_right_hip + 50 <= y_right_knee ):
            count_bool = True

                
    bottomLeftCornerOfText = (10,400)
    fontScale = 2
    fontColor = (225,0,0)
    lineType = 2
    
    cv2.putText(frame,
                'Number of Squats'+' '+str(squat_counter), 
                bottomLeftCornerOfText, 
                1, 
                fontScale,
                fontColor,
                lineType)    
    
    bottomLeftCornerOfText = (10,440)
    cv2.putText(frame,
                'Calories burnt '+' '+str(squat_counter*0.32)+ 'Kcal', 
                bottomLeftCornerOfText, 
                1, 
                fontScale,
                fontColor,
                lineType)    


    return squat_counter, count_bool           